## **Feature:** Pattern Matching

**Names:** Tanat

### **What it does**
[Brief description]

### **Helper Functions**
[List Helper Functions]

In [88]:
# Get API Key
from dotenv import load_dotenv
load_dotenv()
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("OpenAI API Key not found")

# Import libraries
import pandas as pd
pd.set_option('future.no_silent_downcasting', False)
import numpy as np
import math
import re
import datetime
from pint import UnitRegistry
from pint.errors import UndefinedUnitError
ureg = UnitRegistry()

# Langchain imports
from langchain.chat_models import ChatOpenAI  
from langchain.schema import HumanMessage, SystemMessage

**CLASSIFYING PATTERNS**
-
<hr>

**FUNCTIONS**
- classify_object_columns(df, patterns)
- classify_column(series,patterns)

In [89]:
patterns = {
    # Multiselect patterns - "A; B", "A/B", {'X', 'Y', 'Z'}, [1, 2, 3],
    "multiselect": r"^\s*[^,;/|&]+(?:\s*[,;/|&]\s*[^,;/|&]+|\s+(?:and|AND)\s+[^,;/|&]+)+\s*$",
    "multiselect_structured": r"^\s*[\[\{]\s*[^,\]\}]+(?:\s*,\s*[^,\]\}]+)+\s*[\]\}]\s*$",

    # Numerical values
    "numeric_plain": r"^\s*[-+]?(?:\d{1,3}(?:[,.\s]\d{3})*(?:[,.]\d+)?|\d+(?:[,.]\d+)?)\s*$",
    "percentage": r"^\s*[-+]?\d+(?:[,.]\d+)?\s*%\s*$",
    "currency": r"^\s*[$€£¥₹¢₽₦₴₪₩]?\s*[-+]?(?:\d{1,3}(?:[,.\s]\d{3})*(?:[,.]\d+)?|\d+(?:[,.]\d+)?)\s*(?:USD|EUR|AUD|GBP|INR|JPY|CAD|CHF|SEK|NOK|DKK|CNY|KRW|RUB|BRL|MXN)?\s*$",
    "range": r"^(?:[^\d]+)?([-+]?\d+(?:[.,:]?\d+)?)\s*(?:[-–—]\s*|\s+(?:to|TO|bis|à|and|AND)\s+)\s*([-+]?\d+(?:[.,:]?\d+)?)(?:\s+([a-zA-Z ]+))?\s*$",
    "numeric_with_units": r"^\s*(?:\w+(?:\s+\w+)*)?\s*[-+]?\d+(?:[,.]\d+)?\s*(?:[-–—]\s*|\s+(?:to|TO)\s+)\s*[-+]?\d+(?:[,.]\d+)?\s*(?:\w+(?:\s+\w+)*)?\s*$",
    "scientific": r"^\s*[-+]?\d+(?:[,.]\d+)?[eE][-+]?\d+\s*$",   
    
    # Date-time patterns (not  tested)
    "date_like": r"^\s*(?:\d{1,2}[/.-]\d{1,2}[/.-]\d{2,4}|\d{4}[/.-]\d{1,2}[/.-]\d{1,2})\s*$",
    "time_like": r"^\s*(?:\d{1,2}:\d{2}(?::\d{2})?(?:\s*[APap][Mm])?)\s*$",
}

In [90]:
def classify_object_columns(df, patterns):
    col_types = {}
    object_df = df.select_dtypes(include=['object'])
    for col in object_df:
        col_type = classify_column(df[col], patterns)
        if col_type:
            col_types[col] = col_type
    return col_types

In [91]:
def classify_column(series, patterns=patterns):
    """
    Classify an object column into a numeric-like pattern if most values match.
    """
    # Remove null values and convert to string
    vals = series.dropna().astype(str)
    if vals.empty:
        return None
    
    # Get unique values - much more efficient than sampling
    unique_vals = vals.unique()[0:30]
    total_unique = len(unique_vals)

    if total_unique < 10:
        min_support = 0.5
    elif total_unique < 30:
        min_support = 0.6
    else:
        min_support = 0.7

    # Check each pattern
    for pattern_name, pattern_regex in patterns.items():
        matches = 0
        for val in unique_vals:
            if re.match(pattern_regex, val.strip()):
                matches += 1

        # If this pattern matches enough values, return it
        if matches / total_unique >= min_support:
            return pattern_name
    
    return None

**HANDLING NUMERIC COLUMNS**
-
<hr>

**FUNCTIONS**
- clean_numeric_plain(val)
- handle_numeric(series)

In [92]:
def clean_numeric_plain(val):
    s = str(val).strip()

    # Remove spaces, symbols, currency codes, %
    s = re.sub(r'\s+', '', s)
    s = re.sub(r'[$€£¥₹¢₽₦₴₪₩]', '', s)
    s = re.sub(r'(USD|EUR|AUD|GBP|INR|JPY|CAD|CHF|SEK|NOK|DKK|CNY|KRW|RUB|BRL|MXN)', '', s, flags=re.IGNORECASE)
    is_percent = False
    if '%' in s:
        s = s.replace('%', '')
        is_percent = True

    # Handle different number formats: 1.234,56 or 1,234.56
    if re.match(r'^\d{1,3}(\.\d{3})+,\d+$', s):
        s = s.replace('.', '').replace(',', '.')
    elif re.match(r'^\d{1,3}(,\d{3})+(\.\d+)?$', s):
        s = s.replace(',', '')

    # Handle any decimal commas and stray commas
    elif re.match(r'^\d+,\d+$', s):
        s = s.replace(',', '.')
    s = s.replace(',', '')

    # Convert to float
    try:
        num = float(s)
        if is_percent:
            num = num / 100
        return num
    except:
        return s

In [93]:
def handle_numeric(series):
    errors = {}
    cleaned = [clean_numeric_plain(val) for val in series]
    converted = pd.to_numeric(cleaned, errors='coerce')
    converted = pd.Series(converted, index=series.index, name=series.name)
    for idx, (orig, conv) in enumerate(zip(series, converted)):
        if pd.isna(conv) and pd.notna(orig):
            if orig not in errors:
                errors[orig] = 0
            errors[orig] += 1
            converted.iloc[idx] = orig
    return converted, errors

**HANDLING UNIT COLUMNS**
-
<hr>

ureg: pint's unit registry

**FUNCTIONS**
- unit_exists(unit, ureg)
- handle_unit(series)

In [94]:
# Helper function to check if pint unit exists
def unit_exists(unit, ureg):
    try:
        ureg.Unit(unit)
        return True
    except UndefinedUnitError:
        return False

def handle_units(series):
    """
    Normalize all units to the first unit found in the series and return numeric values.
    """
    errors = {}
    ureg = UnitRegistry()    
    # Find the first unit in the series
    target_unit = None
    for text in series.dropna():
        match = re.search(r'(\d+\.?\d*)\s*([a-zA-Z]+)', str(text))
        if match and unit_exists(match.group(2), ureg):
            target_unit = match.group(2)
            break
        else:
            if str(text) not in errors:
                errors[str(text)] = 0
            errors[str(text)] += 1
    
    if not target_unit:
        return series, errors

    # Helper function within to turn unit strings into numbers
    def convert_units(text):
        # Handle no match or null
        if pd.isna(text):
            return None
        match = re.search(r'(\d+\.?\d*)\s*([a-zA-Z]+)', str(text))
        if not match:
            if str(text) not in errors:
                errors[str(text)] = 0
            errors[str(text)] += 1
            return text

        value, unit = match.groups()
        try:
            quantity = ureg.Quantity(float(value), unit)
            converted = quantity.to(target_unit)
            return converted.magnitude
        except Exception as e:
            if str(text) not in errors:
                errors[str(text)] = 0
            errors[str(text)] += 1
            return text
            
    # Apply convert_unit function
    numeric_series = series.apply(convert_units) 
    numeric_series.name = f"{series.name or 'values'}_{target_unit}"
    return numeric_series, errors

**HANDLING RANGE COLUMNS**
-
<hr>

**FUNCTIONS**
- convert_range(val)
- handle_range(series)

In [95]:
def convert_range(val):
    """
    Extracts numeric ranges and units from strings, computes the midpoint.
    Returns a string "{midpoint} {unit}"
    """
    s = str(val).strip()
    # Match the range pattern
    range_regex = r"^(?:[^\d]+)?([-+]?\d+(?:[.,:]?\d+)?)\s*(?:[-–—]\s*|\s+(?:to|TO|bis|à|and|AND)\s+)\s*([-+]?\d+(?:[.,:]?\d+)?)(?:\s+([a-zA-Z ]+))?\s*$"
    match = re.match(range_regex, s)
    if not match:
        return s
    num1, num2, unit = match.group(1), match.group(2), match.group(3)
    # Normalize decimals
    num1 = num1.replace(',', '.').replace(':', '.')
    num2 = num2.replace(',', '.').replace(':', '.')
    try:
        n1 = float(num1)
        n2 = float(num2)
        midpoint = (n1 + n2) / 2
    except:
        return s
    return f"{midpoint} {unit}"

In [96]:
def handle_range(series):
    """
    Applies handle_range to a pandas Series.
    Returns a pandas Series with "{midpoint} {unit}" or np.nan if parsing fails.
    """
    result = []
    errors = {}
    valid_mask = []
    for val in series:
        if val is None or (isinstance(val, float)):
            result.append(val)
            valid_mask.append(False)
            continue
        cleaned = handle_range(val)
        if isinstance(cleaned, str) and cleaned == str(val).strip():
            if val not in errors:
                errors[val] = 0
            errors[val] += 1
            result.append(val)
            valid_mask.append(False)
        else:
            result.append(cleaned)
            valid_mask.append(True)

    result = pd.Series(result)
    result.name = series.name

    valid_series = result[valid_mask]
    converted, unit_errors = handle_units(valid_series)
    result.name = converted.name
    errors.update(unit_errors)

    # Update only valid positions in the result
    result.loc[valid_series.index] = converted
    return result, errors

**CONVERTING COLUMNS**
-
<hr>

**FUNCTIONS**
- classify_and_convert(df, patterns)
- llm_fix_errors(df, all_errors)
- object_conversion_with_llm(df, patterns)

In [97]:
def classify_and_convert(df, patterns):
    """
    Classifies object columns and applies appropriate conversion.
    """
    col_types = classify_object_columns(df,patterns)
    all_errors = {}
    if not col_types:
        print("No patterns detected")
        return df, all_errors

    print("Classified columns, Attempting Inital conversion")
    for col, col_type in col_types.items():
        if col_type in {'multiselect, multiselect_structured'}:
            pass 
        elif col_type == 'range':
            converted, errors = handle_range(df[col])
        elif col_type in {'percentage', 'numeric_plain', 'currency', 'scientific'}:
            converted, errors = handle_numeric(df[col])
        elif col_type == 'numeric_with_units':
            converted, errors = handle_units(df[col])
        else:
            continue
        # Update df column and rename if name changes
        df[col] = converted
        if converted.name != col:
            df.rename(columns={col: converted.name}, inplace=True)
            col_key = converted.name
        else:
            col_key = col
        # Update log with error values
        if errors:
            all_errors[col_key] = {}
            all_errors[col_key]['errors'] = errors
            all_errors[col_key]['type'] = col_type
    print("Initial conversion attempt completed")
    return df, all_errors

In [131]:
def llm_fix_errors(df, all_errors):
    prompt = """
    The following columns have already been attempted to convert to numerical columns.
    Please suggest Python code to fix these by suggesting appropriate numerical values to replace these error values:\n\n
    - Assume dataframe already exists stored in variable 'df'
    - Import pandas as pd and numpy as np at the start

    Rules:
    - For open-ended ranges or qualitative values, use reasonable domain assumptions and try to CALCULATE MIDPOINTS
    (e.g., max working years is 50, max age is 100, etc.) when calculating midpoints or replacements.
    - Print reasonings for your assumptions after any assumptions have been made
    (e.g., print('LLM: Assumed 0.25 hours for less than 30 minutes and 5 hours for over 4')
    - If no assumptions needed don't assume just replace strings with the appropriate numerical value
    - ASSUMING ADDS BIAS TO DATA USE EXACT VALUE WHEN IT MAKES SENSE (NO RANGES)
    (e.g., '59.5 employees' -> 59.5, '3.5 times per week' -> 3.5)
    - Handle units carefully make sure units allign with other units in the column
    - Use this EXACT format to avoid pandas warnings:
    df[col] = df[col].replace({'old_value': new_value}).infer_objects(copy=False)
    - ONLY convert to np.nan if data doesn't make sense/allign with context and unsure on how to convert
    
    Example:
    df['Age'] = df['Age'].replace({'Under 18': 17, '65+': 70}).infer_objects(copy=False)
    print('LLM: Assumed 17 for Under 18 and 70 for 65+ in Age column')
    
    ONLY GENERATE VALID PYTHON CODE (no markdown blocks, no explanations)
    """
    print("Final LLM attempt to fix errors (Might take awhile)")
    errors_context = ""
    df_context = ""
    for col, info in all_errors.items():
        error_type = info.get('type', 'unknown')
        error_values = list(info['errors'].keys())
        errors_context += f"Column: {col}\nType: {error_type}\nErrors: {error_values[:10]}\n\n"

        df_unique = df[col].dropna().astype(str).unique()
        df_unique_len = len(df_unique)
        df_unique_vals = ", ".join(df_unique[:20])
        df_context += f"Column:{col}\nUniques:{df_unique_len}\nValues:{df_unique_vals}\n\n"

    messages = []
    messages.append(SystemMessage(prompt))
    messages.append(HumanMessage(errors_context))
        
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
    response = llm.invoke(messages)
    generated_code = response.content.strip()
    # print(generated_code)
    
    try:
        original_df = df.copy()
        namespace = {"df": df, "pd": pd, "np": np}
        exec(generated_code, namespace)
        df = namespace["df"]
        return df
    except Exception as e:
        print(f"Error: {e}")
        print(f"Generated Code:{generated_code}")
        return original_df

In [ ]:
def object_conversion_with_llm(df, patterns):
    """
    Runs the full object column conversion pipeline:
    1. Classifies and converts object columns.
    2. Uses LLM to fix remaining errors.
    3. Checks which columns are still object dtype.
    """
    # Step 1: Initial classification + conversion 
    df, all_errors = classify_and_convert(df, patterns)
    # Step 2: LLM fixes any errors
    if all_errors:
        failed_cols=[]
        df = llm_fix_errors(df, all_errors)
        # Check for remaining object columns
        for col in all_errors.keys():
            if col in df.columns and df[col].dtype == 'object':
                failed_cols.append(col)
        if not failed_cols:
            print("All error columns have been successfully converted from object dtype.")
        else:
            print(f"The following columns are still object dtype: {failed_cols}")
    return df

In [ ]:
def pattern_matching(df, user_query):
    """
    Main function that gets called by the main router.
    MUST take (user_query, df) and return df
    """
    # TODO: Create helper docs (Reimplement with functions)
    helper_docs = f"""
    VARIABLES AVAILABLE
    - patterns: a dict containing regex patterns in order to classify columns
    contains patterns for: {", ".join(patterns.keys())}
    sample patterns: {', '.join(list(patterns.values())[0:4])}

    FUNCTIONS AVAILABLE:
    - col_type = classify_column(series,patterns)
    takes a singular column and a dict of regex patterns then classifies them 
    returns a string, column type matched from patterns dict

    - col_types = classify_object_columns(df, patterns)
    automatically takes object columns from a df and classfies them with classify_column
        returns 
        - a dict of matched column names and its column types e.g. {{'Smoke everyday': 'percentage', 'Company Size': 'range'}}
    MAKE SURE TO PRINT TO SEE RESULTS
        
    - df = object_conversion_with_llm(df, patterns):
    runs the full pipline classify_and_convert and takes any remaining string values 
    that couldn't be converted and gets llm to process and suggest numerical replacement values to convert column 
    returns:
        -converted df
    """
    
    # Create message chain
    messages = []
    messages.append(SystemMessage(content=f"""
    You are a data cleaning agent.
    
    Dataset info: Shape: {df.shape}, Sample: {df.head(3).to_string()}
    
    Helper functions available:
    {helper_docs}

    Libraries available:
    - pd (pandas), np (numpy)
    - math, re, datetime
    - preprocessing, impute (from sklearn)
    
    Rules:
    - Return only executable Python code, no explanations, no markdown blocks
    - Use helper functions where possible
    - Store final result in 'df'
    - No explanations, just code
    """))
    messages.append(HumanMessage(content=f"User request: {user_query}"))
    
    # Call LLM with message chain
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
    response = llm.invoke(messages)
    generated_code = response.content.strip()
    # Execute code
    try:
        original_df = df.copy()
        exec(generated_code)
        return df
    except Exception as e:
        print(f"Error: {e}")
        print(f"Generated Code:{generated_code}")
        return original_df

**TESTING**
-

In [139]:
# df = pd.read_csv("../sample_data/smoke.csv")[0:5000]
df = pd.read_csv("../sample_data/smoke.csv")[0:5000]

In [142]:
# Sample Queries:
# 1. classify columns
test_df = pattern_matching(df, "Detect formatting issues")

{'Smoke everyday': 'percentage', 'Smoke some days': 'percentage', 'Former smoker': 'percentage', 'Never smoked': 'percentage'}
